In [102]:
import numpy as np
import pandas as pd
import os
from functools import reduce
from natsort import natsorted

In [103]:
df1 = pd.read_csv('../Data/Clean/USAFacts/COVIDcounts_county_clean.csv', index_col = 0)
print(df1.shape)
df1.head()

(3142, 895)


,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,5813,5818,5825,5832,5832,5832,5857,5863,5880,5891
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,6630,6648,6654,6663,6663,6663,6679,6687,6700,6704
5,1009,Blount County,AL,1,0,0,0,0,0,0,...,15246,15249,15272,15283,15283,15283,15324,15335,15351,15370


In [104]:
for index, column in enumerate(df1.columns[:]):
    if column.startswith('2020-12-30'):
        labs20 = column
    if column.startswith('2021-12-30'):
        labs21 = column
    if column.startswith('2022-06-30'):
        labs22 = column

print(labs20)
print(labs21)
print(labs22)

2020-12-30
2021-12-30
2022-06-30


In [105]:
df1 = pd.DataFrame(np.repeat(df1.values, 3, axis = 0), columns = df1.columns)
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905


In [106]:
df1['year'] = np.nan
#df1['year'] = df1.year.astype('int64').fillna()
df1['year'] = pd.to_numeric(df1['year'], errors = 'coerce')
for index in df1.index:
    if index % 3 == 0:
        df1.iloc[index, -1] = 2020
    elif index % 3 == 1:
        df1.iloc[index, -1] = 2021
    else:
        df1.iloc[index, -1] = 2022
df1['year'] = df1['year'].astype('int64')
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2020
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2021
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2022
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58169,58293,58372,58372,58372,58595,58695,58796,58905,2020
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58169,58293,58372,58372,58372,58595,58695,58796,58905,2021


In [107]:
df1['covid_cases'] = np.nan
#df1['covid_cases'] = df1['covid_cases'].astype('int64').fillna(np.nan)
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year,covid_cases
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2020,NaN
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2021,NaN
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2022,NaN
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2020,NaN
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2021,NaN


In [108]:
for index in df1.index:
    if index % 3 == 0:
        df1.iloc[index,-1] = df1.loc[index,labs20]
    elif index % 3 == 1:
        df1.iloc[index,-1] = df1.loc[index,labs21]
    else:
        df1.iloc[index,-1] = df1.loc[index,labs22]
df1

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year,covid_cases
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2020,4164.0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2021,10953.0
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2022,16536.0
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2020,13392.0
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2021,39583.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9421,56043,Washakie County,WY,56,0,0,0,0,0,0,...,2490,2490,2490,2490,2490,2482,2482,2482,2021,1880.0
9422,56043,Washakie County,WY,56,0,0,0,0,0,0,...,2490,2490,2490,2490,2490,2482,2482,2482,2022,2482.0
9423,56045,Weston County,WY,56,0,0,0,0,0,0,...,1641,1641,1641,1641,1641,1643,1643,1643,2020,475.0
9424,56045,Weston County,WY,56,0,0,0,0,0,0,...,1641,1641,1641,1641,1641,1643,1643,1643,2021,1254.0


In [109]:
df = df1[['year', 'fips','covid_cases', 'county_name']]
print(df.dtypes)
df[['covid_cases']] = df[['covid_cases']].apply(pd.to_numeric)
print(df.dtypes)
df

year             int64
fips            object
covid_cases    float64
county_name     object
dtype: object
year             int64
fips            object
covid_cases    float64
county_name     object
dtype: object


/var/folders/jb/2grr3hjx7fng_kz01_rqpskm0000gn/T/ipykernel_16007/2871141585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['covid_cases']] = df[['covid_cases']].apply(pd.to_numeric)


,year,fips,covid_cases,county_name
0,2020,1001,4164.0,Autauga County
1,2021,1001,10953.0,Autauga County
2,2022,1001,16536.0,Autauga County
3,2020,1003,13392.0,Baldwin County
4,2021,1003,39583.0,Baldwin County
...,...,...,...,...
9421,2021,56043,1880.0,Washakie County
9422,2022,56043,2482.0,Washakie County
9423,2020,56045,475.0,Weston County
9424,2021,56045,1254.0,Weston County


In [110]:
df.to_csv('../Data/Intermediate/COVIDcounty_cases_processed.csv')

In [111]:
dp03, dp05 = [], []
path = '../Data/Clean/ACS'
for file in natsorted(os.listdir(path)):
    if file.find('DP03') != -1:
        print(file)
        dp03.append(pd.read_csv(os.path.join(path, file), index_col = 0))
    if file.find('DP05') != -1:
        print(file)
        dp05.append(pd.read_csv(os.path.join(path, file), index_col = 0))

i = 0
while i != len(dp03):
    dp03[i].drop(columns = ['geographic_area_name','id'], inplace = True)
    print('03: ', dp03[i].shape)
    i += 1

ACSDP032010_clean.csv
ACSDP032011_clean.csv
ACSDP032012_clean.csv
ACSDP032013_clean.csv
ACSDP032014_clean.csv
ACSDP032015_clean.csv
ACSDP032016_clean.csv
ACSDP032017_clean.csv
ACSDP032018_clean.csv
ACSDP032019_clean.csv
ACSDP032020_clean.csv
ACSDP052010_clean.csv
ACSDP052011_clean.csv
ACSDP052012_clean.csv
ACSDP052013_clean.csv
ACSDP052014_clean.csv
ACSDP052015_clean.csv
ACSDP052016_clean.csv
ACSDP052017_clean.csv
ACSDP052018_clean.csv
ACSDP052019_clean.csv
ACSDP052020_clean.csv
03:  (3221, 20)
03:  (3221, 20)
03:  (3221, 20)
03:  (3221, 20)
03:  (3220, 20)
03:  (3220, 20)
03:  (3220, 20)
03:  (3220, 20)
03:  (3220, 20)
03:  (3220, 20)
03:  (3221, 20)


In [112]:
yrs = range(2010, 2021)
yrs_rate = [1.36, 1.31, 1.29, 1.27, 1.24, 1.24, 1.23, 1.21, 1.18, 1.16, 1.15]

print(len(yrs))
print(len(yrs_rate))
print(len(dp03))
for index, i in enumerate(dp03):
    print(yrs_rate[index])
    #dp03['fips']= dp03['fips'].astype('int64')
    i['per_capita_income'] = i['per_capita_income_' + str(yrs[index])].astype('float64')
    #print(index, ':    ', i.dtypes)
    print(index, ': ', yrs_rate[index])
    mult = yrs_rate[index]
    i['per_capita_income_2022'] = i['per_capita_income'] * mult
    i.drop(columns = ['per_capita_income_' + str(yrs[index]), 'per_capita_income'], inplace = True)
    
#dp03

11
11
11
1.36
0 :  1.36
1.31
1 :  1.31
1.29
2 :  1.29
1.27
3 :  1.27
1.24
4 :  1.24
1.24
5 :  1.24
1.23
6 :  1.23
1.21
7 :  1.21
1.18
8 :  1.18
1.16
9 :  1.16
1.15
10 :  1.15


In [113]:
df03 = pd.concat(dp03, axis = 0).sort_values(by = ['fips', 'year']).reset_index(drop = True)
df05 = pd.concat(dp05, axis = 0).sort_values(by = ['fips', 'year']).reset_index(drop = True)

In [114]:
print(df05['fips'].nunique())
print(df03['fips'].nunique())
print(df05['year'].value_counts())
print(df03['year'].value_counts())

3225
3225
2010    3221
2011    3221
2012    3221
2013    3221
2020    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: year, dtype: int64
2010    3221
2011    3221
2012    3221
2013    3221
2020    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: year, dtype: int64


In [115]:
print(df03.shape)
print(df05.shape)

(35425, 20)
(35425, 11)


In [116]:
df03['health_insurance_rate'] = pd.to_numeric(df03['health_insurance_rate'], errors = 'coerce')
#df05['median_age'] = pd.to_numeric(df05['median_age'], errors = 'coerce')
print(df03.dtypes)
print(df05.dtypes)
df03[df03['fips'] == 45001]['unemployment_rate']

unemployment_rate                                                                                  float64
employment_pop_ratio                                                                               float64
pct_ind_agriculture_forestry_fishing_and_hunting_and_mining                                        float64
pct_ind_construction                                                                               float64
pct_ind_manufacturing                                                                              float64
pct_ind_wholesale_trade                                                                            float64
pct_ind_retail_trade                                                                               float64
pct_ind_transportation_and_warehousing_and_utilities                                               float64
pct_ind_information                                                                                float64
pct_ind_finance_and_insurance_and_rea

25477     8.2
25478     8.0
25479     7.5
25480     7.9
25481     6.6
25482    10.7
25483     9.4
25484     9.4
25485     7.6
25486     5.5
25487     5.0
Name: unemployment_rate, dtype: float64

In [117]:
df03.to_csv('../Data/Intermediate/ACSDP03_merge.csv')
df05.to_csv('../Data/Intermediate/ACSDP05_merge.csv')

In [118]:
hps = pd.read_csv('../Data/Clean/HPS/hps_clean.csv', index_col = 0)
print(hps.columns)

Index(['year', 'area', 'pct_delayed_healthcare'], dtype='object')


In [119]:
hps['state_fips'] = np.nan
state = [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 
         21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 
         38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56]
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming']

dic = {state[i]: us_states[i] for i in range(len(state))}

print(len(state))
print(len(us_states))
print(dic)

hps['state_fips'] = dic
hps

51
51
{1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware', 11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska', 32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota', 47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington', 54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming'}


,year,area,pct_delayed_healthcare,state_fips
0,2020,Alabama,39.200000,1
1,2020,Alaska,45.708333,2
2,2020,Arizona,41.016667,4
3,2020,Arkansas,40.575000,5
4,2020,California,41.116667,6
5,2020,Colorado,41.158333,8
6,2020,Connecticut,41.391667,9
7,2020,Delaware,38.766667,10
8,2020,District of Columbia,43.500000,11
9,2020,Florida,40.216667,12


In [120]:
hps.to_csv('../Data/Intermediate/hps_processed.csv')

In [189]:
path3 = '../Data/Clean/nonACSDemographics'
saipes = []
for file in natsorted(os.listdir(path3)):
    if file.startswith('S'):
        saipes.append(pd.read_csv(os.path.join(path3, file), sep = ',', index_col = 0))
        print(file)

SAIPE2002_clean.csv
SAIPE2003_clean.csv
SAIPE2004_clean.csv
SAIPE2005_clean.csv
SAIPE2006_clean.csv
SAIPE2007_clean.csv
SAIPE2008_clean.csv
SAIPE2009_clean.csv


In [190]:
yrs = range(2002,2010)
yr_rates = [1.65, 1.61, 1.56, 1.52, 1.46, 1.42, 1.35, 1.37]

for index, i in enumerate(saipes):
    #i['poverty_rate'] = pd.to_numeric(i['poverty_rate'], errors = 'coerce')
    i.replace('.', np.nan, inplace=True)
    #if 'poverty_rate' in i.columns: 
        #i['poverty_rate'] = i['poverty_rate'].astype('float64').fillna(np.nan)
    #i.iloc[:,1] = i.iloc[:,1].astype('float64').fillna(np.nan)
    
    i['fips']= i['fips'].astype('int64')
    i['per_capita_income'] = i['per_capita_income_' + str(yrs[index])].astype('float64')
    print(index, ':    ', i.dtypes)
    i['per_capita_income_2022'] = i['per_capita_income'] * yr_rates[index]
    i.drop(columns = ['per_capita_income_' + str(yrs[index]), 'per_capita_income'], inplace = True)

    print(index, ':    ', i.dtypes)

0 :     poverty_rate              float64
per_capita_income_2002     object
fips                        int64
year                        int64
per_capita_income         float64
dtype: object
0 :     poverty_rate              float64
fips                        int64
year                        int64
per_capita_income_2022    float64
dtype: object
1 :     poverty_rate              float64
per_capita_income_2003     object
fips                        int64
year                        int64
per_capita_income         float64
dtype: object
1 :     poverty_rate              float64
fips                        int64
year                        int64
per_capita_income_2022    float64
dtype: object
2 :     poverty_rate               object
per_capita_income_2004     object
fips                        int64
year                        int64
per_capita_income         float64
dtype: object
2 :     poverty_rate               object
fips                        int64
year                        int6

In [191]:
df = pd.concat(saipes, axis = 0)
df = df.sort_values(by = ['fips', 'year']).reset_index(drop = True)
print(df.shape)
df.head(20)
df.to_csv('../Data/Intermediate/saipe0210.csv')

(25131, 4)


In [192]:
path3 = '../Data/Clean/nonACSDemographics'
other = []
for file in natsorted(os.listdir(path3)):
    if not file.startswith(('S', '.')):
        other.append(pd.read_csv(os.path.join(path3, file), sep = ',', index_col = 0))
        print(file)
        
for index, i in enumerate(other):
    i = i[(i['year']<=2009) & (i['year']>=2002)]

census_race_sex_ethnicity_pop_2000_2010_clean.csv
census_total_pop_2000_2010_clean.csv
usdaERSCountyEmployment_clean.csv


In [193]:
cov02 = other[0].columns.intersection(other[1].columns)
df02 = pd.merge(other[0], other[1], on = cov02.to_list(), how = 'outer')
df02 = df02[(df02['year'] >= 2002) & (df02['year'] <= 2020)]
df02 = df02.sort_values(by = ['fips','year']).reset_index(drop = True)
df02.head(25)

,year,fips,pct_asian,pct_black,pct_native_american,pct_native_hawaiian_pacific_islander,pct_two_or_more_races,pct_white,pct_hispanic,pct_female,total_pop
0,2002,1001,0.006055,0.172123,0.004792,0.000283,0.009301,0.807445,0.016119,0.512906,45909
1,2003,1001,0.006282,0.172842,0.004808,0.000321,0.009850,0.805897,0.016923,0.513675,46800
2,2004,1001,0.006740,0.173531,0.004673,0.000351,0.010565,0.804139,0.017967,0.513873,48366
3,2005,1001,0.007247,0.173706,0.004549,0.000745,0.011313,0.802440,0.019869,0.512843,49676
4,2006,1001,0.007286,0.176181,0.004481,0.000760,0.011904,0.799388,0.020320,0.513170,51328
5,2007,1001,0.007709,0.177273,0.004446,0.000725,0.012518,0.797328,0.022193,0.514016,52405
6,2008,1001,0.008033,0.178126,0.004730,0.000845,0.013195,0.795071,0.022055,0.514368,53277
7,2009,1001,0.008497,0.179071,0.004674,0.000868,0.013799,0.793091,0.023404,0.512995,54135
8,2002,1003,0.004549,0.101624,0.006252,0.000331,0.009327,0.877917,0.021952,0.510675,147957
9,2003,1003,0.004719,0.100555,0.006475,0.000396,0.009848,0.878007,0.024084,0.510247,151509


In [194]:
for i in other[2:3]:
    cov = df02.columns.intersection(i.columns)
    df02 = pd.merge(df02, i, on = cov.to_list(), how = 'outer')
    df02 = df02[(df02['year'] >= 2002) & (df02['year'] <= 2020)]
    df02 = df02.sort_values(by = ['fips','year']).reset_index(drop = True)
    print(df02['year'].value_counts())
    print(i.dtypes)
df02.head(25)

2002    3228
2003    3228
2004    3228
2005    3228
2006    3228
2007    3228
2008    3228
2009    3228
Name: year, dtype: int64
fips                   int64
year                   int64
employed_persons     float64
unemployment_rate    float64
dtype: object


,year,fips,pct_asian,pct_black,pct_native_american,pct_native_hawaiian_pacific_islander,pct_two_or_more_races,pct_white,pct_hispanic,pct_female,total_pop,employed_persons,unemployment_rate
0,2002,1001,0.006055,0.172123,0.004792,0.000283,0.009301,0.807445,0.016119,0.512906,45909.0,21096.0,4.8
1,2003,1001,0.006282,0.172842,0.004808,0.000321,0.009850,0.805897,0.016923,0.513675,46800.0,21557.0,5.0
2,2004,1001,0.006740,0.173531,0.004673,0.000351,0.010565,0.804139,0.017967,0.513873,48366.0,22146.0,4.7
3,2005,1001,0.007247,0.173706,0.004549,0.000745,0.011313,0.802440,0.019869,0.512843,49676.0,22986.0,3.8
4,2006,1001,0.007286,0.176181,0.004481,0.000760,0.011904,0.799388,0.020320,0.513170,51328.0,23619.0,3.3
5,2007,1001,0.007709,0.177273,0.004446,0.000725,0.012518,0.797328,0.022193,0.514016,52405.0,23610.0,3.4
6,2008,1001,0.008033,0.178126,0.004730,0.000845,0.013195,0.795071,0.022055,0.514368,53277.0,23376.0,5.3
7,2009,1001,0.008497,0.179071,0.004674,0.000868,0.013799,0.793091,0.023404,0.512995,54135.0,22464.0,8.9
8,2002,1003,0.004549,0.101624,0.006252,0.000331,0.009327,0.877917,0.021952,0.510675,147957.0,65881.0,5.0
9,2003,1003,0.004719,0.100555,0.006475,0.000396,0.009848,0.878007,0.024084,0.510247,151509.0,69010.0,4.9


In [195]:
df02.to_csv('../Data/Intermediate/other0209.csv')

In [196]:
covc = df.columns.intersection(df02.columns)
comp = pd.merge(df, df02, on = covc.to_list(), how = 'outer')
comp.sort_values(by = ['fips', 'year']).head(25)
comp.to_csv('../Data/Intermediate/total0209.csv')

In [163]:
cov1 = df.columns.intersection(other[0].columns)
df1 = pd.merge(df, other[0], on = cov1.to_list(), how = 'outer')
print(df1.columns)
df1.head(20)

Index(['poverty_rate', 'fips', 'year', 'per_capita_income_2022', 'pct_asian',
       'pct_black', 'pct_native_american',
       'pct_native_hawaiian_pacific_islander', 'pct_two_or_more_races',
       'pct_white', 'pct_hispanic', 'pct_female'],
      dtype='object')


,poverty_rate,fips,year,per_capita_income_2022,pct_asian,pct_black,pct_native_american,pct_native_hawaiian_pacific_islander,pct_two_or_more_races,pct_white,pct_hispanic,pct_female
0,10.3,1001,2002,70687.65,0.006055,0.172123,0.004792,0.000283,0.009301,0.807445,0.016119,0.512906
1,10.4,1001,2003,71228.01,0.006282,0.172842,0.004808,0.000321,0.009850,0.805897,0.016923,0.513675
2,11.6,1001,2004,70791.24,0.006740,0.173531,0.004673,0.000351,0.010565,0.804139,0.017967,0.513873
3,10.4,1001,2005,68428.88,0.007247,0.173706,0.004549,0.000745,0.011313,0.802440,0.019869,0.512843
4,12.5,1001,2006,67876.86,0.007286,0.176181,0.004481,0.000760,0.011904,0.799388,0.020320,0.513170
5,10.4,1001,2007,71532.50,0.007709,0.177273,0.004446,0.000725,0.012518,0.797328,0.022193,0.514016
6,10.7,1001,2008,69689.70,0.008033,0.178126,0.004730,0.000845,0.013195,0.795071,0.022055,0.514368
7,11.2,1001,2009,72720.97,0.008497,0.179071,0.004674,0.000868,0.013799,0.793091,0.023404,0.512995
8,10.7,1003,2002,67349.70,0.004549,0.101624,0.006252,0.000331,0.009327,0.877917,0.021952,0.510675
9,10.1,1003,2003,67557.21,0.004719,0.100555,0.006475,0.000396,0.009848,0.878007,0.024084,0.510247


In [164]:
cov2 = df1.columns.intersection(other[1].columns)
df2 = pd.merge(df1, other[1], on = cov2.to_list(), how = 'outer')
print(df2.columns)
print(df2.shape)
df2.head(20)

Index(['poverty_rate', 'fips', 'year', 'per_capita_income_2022', 'pct_asian',
       'pct_black', 'pct_native_american',
       'pct_native_hawaiian_pacific_islander', 'pct_two_or_more_races',
       'pct_white', 'pct_hispanic', 'pct_female', 'total_pop'],
      dtype='object')
(31450, 13)


,poverty_rate,fips,year,per_capita_income_2022,pct_asian,pct_black,pct_native_american,pct_native_hawaiian_pacific_islander,pct_two_or_more_races,pct_white,pct_hispanic,pct_female,total_pop
0,10.3,1001,2002,70687.65,0.006055,0.172123,0.004792,0.000283,0.009301,0.807445,0.016119,0.512906,45909.0
1,10.4,1001,2003,71228.01,0.006282,0.172842,0.004808,0.000321,0.009850,0.805897,0.016923,0.513675,46800.0
2,11.6,1001,2004,70791.24,0.006740,0.173531,0.004673,0.000351,0.010565,0.804139,0.017967,0.513873,48366.0
3,10.4,1001,2005,68428.88,0.007247,0.173706,0.004549,0.000745,0.011313,0.802440,0.019869,0.512843,49676.0
4,12.5,1001,2006,67876.86,0.007286,0.176181,0.004481,0.000760,0.011904,0.799388,0.020320,0.513170,51328.0
5,10.4,1001,2007,71532.50,0.007709,0.177273,0.004446,0.000725,0.012518,0.797328,0.022193,0.514016,52405.0
6,10.7,1001,2008,69689.70,0.008033,0.178126,0.004730,0.000845,0.013195,0.795071,0.022055,0.514368,53277.0
7,11.2,1001,2009,72720.97,0.008497,0.179071,0.004674,0.000868,0.013799,0.793091,0.023404,0.512995,54135.0
8,10.7,1003,2002,67349.70,0.004549,0.101624,0.006252,0.000331,0.009327,0.877917,0.021952,0.510675,147957.0
9,10.1,1003,2003,67557.21,0.004719,0.100555,0.006475,0.000396,0.009848,0.878007,0.024084,0.510247,151509.0


In [165]:
df2.columns

Index(['poverty_rate', 'fips', 'year', 'per_capita_income_2022', 'pct_asian',
       'pct_black', 'pct_native_american',
       'pct_native_hawaiian_pacific_islander', 'pct_two_or_more_races',
       'pct_white', 'pct_hispanic', 'pct_female', 'total_pop'],
      dtype='object')

In [166]:
cov3 = df.columns.intersection(other[2].columns)
df3 = pd.merge(df2, other[2], on = cov2.to_list(), how = 'outer')
print(df3.columns)
print(df3.shape)
df3.head(20)

Index(['poverty_rate', 'fips', 'year', 'per_capita_income_2022', 'pct_asian',
       'pct_black', 'pct_native_american',
       'pct_native_hawaiian_pacific_islander', 'pct_two_or_more_races',
       'pct_white', 'pct_hispanic', 'pct_female', 'total_pop',
       'employed_persons', 'unemployment_rate'],
      dtype='object')
(32280, 15)


,poverty_rate,fips,year,per_capita_income_2022,pct_asian,pct_black,pct_native_american,pct_native_hawaiian_pacific_islander,pct_two_or_more_races,pct_white,pct_hispanic,pct_female,total_pop,employed_persons,unemployment_rate
0,10.3,1001,2002,70687.65,0.006055,0.172123,0.004792,0.000283,0.009301,0.807445,0.016119,0.512906,45909.0,21096.0,4.8
1,10.4,1001,2003,71228.01,0.006282,0.172842,0.004808,0.000321,0.009850,0.805897,0.016923,0.513675,46800.0,21557.0,5.0
2,11.6,1001,2004,70791.24,0.006740,0.173531,0.004673,0.000351,0.010565,0.804139,0.017967,0.513873,48366.0,22146.0,4.7
3,10.4,1001,2005,68428.88,0.007247,0.173706,0.004549,0.000745,0.011313,0.802440,0.019869,0.512843,49676.0,22986.0,3.8
4,12.5,1001,2006,67876.86,0.007286,0.176181,0.004481,0.000760,0.011904,0.799388,0.020320,0.513170,51328.0,23619.0,3.3
5,10.4,1001,2007,71532.50,0.007709,0.177273,0.004446,0.000725,0.012518,0.797328,0.022193,0.514016,52405.0,23610.0,3.4
6,10.7,1001,2008,69689.70,0.008033,0.178126,0.004730,0.000845,0.013195,0.795071,0.022055,0.514368,53277.0,23376.0,5.3
7,11.2,1001,2009,72720.97,0.008497,0.179071,0.004674,0.000868,0.013799,0.793091,0.023404,0.512995,54135.0,22464.0,8.9
8,10.7,1003,2002,67349.70,0.004549,0.101624,0.006252,0.000331,0.009327,0.877917,0.021952,0.510675,147957.0,65881.0,5.0
9,10.1,1003,2003,67557.21,0.004719,0.100555,0.006475,0.000396,0.009848,0.878007,0.024084,0.510247,151509.0,69010.0,4.9


In [132]:
df.head(20)

,poverty_rate,fips,year,per_capita_income_2022
0,10.3,1001,2002,70687.65
1,10.4,1001,2003,71228.01
2,11.6,1001,2004,70791.24
3,10.4,1001,2005,68428.88
4,12.5,1001,2006,67876.86
5,10.4,1001,2007,71532.50
6,10.7,1001,2008,69689.70
7,11.2,1001,2009,72720.97
8,10.7,1003,2002,67349.70
9,10.1,1003,2003,67557.21


In [133]:
df.dtypes

poverty_rate               object
fips                        int64
year                        int64
per_capita_income_2022    float64
dtype: object

In [134]:
cdc_age = pd.read_csv('../Data/Clean/CDC/CDCSTDagesex20192020_clean.csv', index_col = 0)
cdc_age.columns =cdc_age.columns.str.replace('unknown', 'unknown_age')
list(cdc_age.columns)
cdc_age.to_csv('../Data/Clean/CDC/CDCSTDagesex20192020_clean.csv')

In [135]:
cdc_race = pd.read_csv('../Data/Intermediate/CDCSTDoverall2000_2020_processed.csv', index_col = 0)
cdc_race.columns = cdc_race.columns.str.replace('unknown', 'unknown_race')
list(cdc_race.columns)
#cdc_race.to_csv('../Data/Intermediate/CDCSTDoverall2000_2020_processed.csv')

['year',
 'fips',
 'cases_chlamydia',
 'cases_early_non-primary_non-secondary_syphilis',
 'cases_gonorrhea',
 'cases_primary_and_secondary_syphilis',
 'rate_per_100000_chlamydia',
 'rate_per_100000_early_non-primary_non-secondary_syphilis',
 'rate_per_100000_gonorrhea',
 'rate_per_100000_primary_and_secondary_syphilis',
 'cases_female_chlamydia_american_indian_alaska_native',
 'cases_female_chlamydia_asian',
 'cases_female_chlamydia_black_african_american',
 'cases_female_chlamydia_hispanic_latino',
 'cases_female_chlamydia_multiracial',
 'cases_female_chlamydia_native_hawaiian_other_pacific_islander',
 'cases_female_chlamydia_unknown_race',
 'cases_female_chlamydia_white',
 'cases_female_early_non-primary,_non-secondary_syphilis_american_indian_alaska_native',
 'cases_female_early_non-primary,_non-secondary_syphilis_asian',
 'cases_female_early_non-primary,_non-secondary_syphilis_black_african_american',
 'cases_female_early_non-primary,_non-secondary_syphilis_hispanic_latino',
 'case